## What metaclasses are – quick recap
from https://breadcrumbscollector.tech/when-to-use-metaclasses-in-python-5-interesting-use-cases/

They are classes for classes (hence “meta” in their name).

Simply saying – while classes are blueprints for objects, metaclasses are blueprints for classes. Class acts as a blueprint when we create an instance of it whereas metaclass acts as a blueprint only when a class is defined.

In python everything is an object. class is also object

class is a blueprint of object, and metaclass is a blueprint of class

<img>https://files.realpython.com/media/class-chain.5cb031a299fe.png</img>

In [25]:
class Test:
    def set_attr(self, attr):
        self.attr = attr
    def get_attr(self):
        return self.attr
    pass

t = Test()

print(t)            # object
print(type(t))      # class
print(type(Test))   # type
print(type(type))   # type

<class '__main__.Test'>
<class 'type'>
<class 'type'>


In the above case:
- t is an instance of class Test.
- Test is an instance of the type metaclass.
- type is also an instance of the type metaclass, so it is an instance of itself.

If type return class.
so you can create class by passing arg: type(name, bases, dict, **kwds) -> a new type

In [ ]:
#type(t) return class, so you can create object from
a = type(t)()       # equal to a = Test()
a.set_attr('a')
print(a.get_attr())

In [26]:
# class is implement form type
Test = type('Test', (), {})

print(Test)         # class

<class '__main__.Test'>


For metaclass it is inherite form type metaclass

In [45]:
# inherit from type
class MyMeta(type):
    # __new__ is a classmethod, even without @classmethod decorator
    def __new__(cls, name, bases, attrs):
        # cls - MyMeta
        # name - name of the class being defined (MyClass in this example)
        # bases - base classes for constructed class, empty tuple in this case
        # attrs - dict with methods and fields defined in class i.e. - {'x': 3}         
        print('cls: ' + str(cls))
        print('name: ' + str(name))
        print('bases: ' + str(bases))
        print('attrs: ' + str(attrs))
        # super().__new__ just returns a new class
        return super().__new__(cls, name, bases, attrs)
    
class ClassFromMeta(metaclass=MyMeta):
    pass
    
ClassFromMeta()

cls: <class '__main__.MyMeta'>
name: ClassFromMeta
bases: ()
attrs: {'__module__': '__main__', '__qualname__': 'ClassFromMeta'}


Metaclass use cases
1. Avoiding decorators repetition or decorating all subclasses
2. Validation of subclasses
3. Registering subclasses – extendable strategy pattern

### 1. Avoiding decorators repetition or decorating all subclasses

In [42]:
def decor(cls):
    print('decorated')
    return cls

class EventMeta(type):
    def __new__(cls, name, bases, namespace):
        new_cls = super().__new__(cls, name, bases, namespace)
        return decor(new_cls)


class Event(metaclass=EventMeta):
    pass


class Event1(Event):
    pass


class Event2(Event):
    pass


# Every class inherite from Event is decorated by @decor and is EventMeta
print(type(Event1))

decorated
decorated
decorated
<class '__main__.EventMeta'>


### 2. Validation of subclasses

In [50]:
import abc

class JsonExporterMeta(abc.ABCMeta):
    _filenames = set()

    def __new__(cls, name, bases, namespace):
        # first execute abc logic
        new_cls = super().__new__(cls, name, bases, namespace)

        if not isinstance(namespace['_filename'], str):
            raise TypeError(f'_filename attribute of {name} class has to be string!')

        if not namespace['_filename'].endswith('.json'):
            raise ValueError(f'_filename attribute of {name} class has to end with ".json"!')

        if namespace['_filename'] in cls._filenames:
            raise ValueError(f'_filename attribute of {name} class is not unique!')

        # ABCMeta register _filenames for check later, this applied to all inheritant class
        cls._filenames.add(namespace['_filename'])
        # If everything ok, return new_cls
        return new_cls

In [65]:
from typing import Dict
import inspect

class StrategyMeta(abc.ABCMeta):
    """
    We keep a mapping of externally used names to classes.
    """
    registry: Dict[str, 'Strategy'] = {}

    def __new__(cls, name, bases, namespace):
        new_cls = super().__new__(cls, name, bases, namespace)

        """
        We register each concrete class
        """
        if not inspect.isabstract(new_cls):
            cls.registry[new_cls.name] = new_cls

        return new_cls 


class Strategy(metaclass=StrategyMeta):
    @property
    @abc.abstractmethod
    def name(self):
        pass

    @abc.abstractmethod
    def validate_credentials(self, login: str, password: str) -> bool:
        pass

    @classmethod
    def for_name(cls, name: str) -> 'Strategy':
        """
        We use registry to build a better class
        """
        return StrategyMeta.registry[name]()
    
    
class AlwaysOk(Strategy):
    name = 'always_ok'

    # This implement abstract method -> concrete method , then register to StrategyMeta.registry
    def validate_credentials(self, login: str, password: str) -> bool:
        return True
    

class AnotherOk(Strategy):
    name = 'another_ok'

    def validate_credentials(self, login: str, password: str) -> bool:
        return True


# example
Strategy.for_name('always_ok').validate_credentials('john', 'x')
Strategy.for_name('always_ok').validate_credentials('jane', 'y')

True

In [66]:
StrategyMeta.registry

{'always_ok': __main__.AlwaysOk, 'another_ok': __main__.AnotherOk}